In [20]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [21]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Miracema Do Tocantins,BR,-9.56,-48.40,73.36,88,90,2.73,light rain,2.93,0.0
1,1,Hermanus,ZA,-34.42,19.23,60.01,93,9,1.99,clear sky,0.00,0.0
2,2,Busselton,AU,-33.65,115.33,70.00,61,95,5.01,light rain,0.13,0.0
3,3,Mount Gambier,AU,-37.83,140.77,62.60,55,75,19.46,broken clouds,0.00,0.0
4,4,Denpasar,ID,-8.65,115.22,86.00,84,20,5.82,few clouds,0.00,0.0


In [22]:
city_data_df.dtypes

City_ID                       int64
City                         object
Country                      object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preference = input("Do you want it to be raining? (yes/no) ")
snow_preference = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [24]:
temp_condition = (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)

if rain_preference == "yes":    
    rain_condition = city_data_df["Rain inches (last 3 hrs)"] > 0
else:
    rain_condition = city_data_df["Rain inches (last 3 hrs)"] == 0


if snow_preference == "yes":    
    snow_condition = city_data_df["Snow inches (last 3 hrs)"] > 0
else:
    snow_condition = city_data_df["Snow inches (last 3 hrs)"] == 0

    
preferred_cities_df = city_data_df.loc[rain_condition & snow_condition & temp_condition] 

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
39,39,Kieta,PG,-6.22,155.63,83.73,74,95,3.91,light rain,1.66,0.0
65,65,Butaritari,KI,3.07,172.79,83.77,70,26,7.83,light rain,1.01,0.0
72,72,Kavieng,PG,-2.57,150.80,84.33,73,30,5.17,light rain,0.43,0.0
87,87,Sinnamary,GF,5.38,-52.95,78.67,86,100,14.65,moderate rain,5.45,0.0
102,102,Cayenne,GF,4.93,-52.33,77.00,100,100,10.09,moderate rain,5.95,0.0


In [25]:
preferred_cities_df.count()

City_ID                     41
City                        41
Country                     41
Lat                         41
Lng                         41
Max Temp                    41
Humidity                    41
Cloudiness                  41
Wind Speed                  41
Current Description         41
Rain inches (last 3 hrs)    41
Snow inches (last 3 hrs)    41
dtype: int64

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng","Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
39,Kieta,PG,83.73,-6.22,155.63,light rain,
65,Butaritari,KI,83.77,3.07,172.79,light rain,
72,Kavieng,PG,84.33,-2.57,150.80,light rain,
87,Sinnamary,GF,78.67,5.38,-52.95,moderate rain,
102,Cayenne,GF,77.00,4.93,-52.33,moderate rain,
119,Faanui,PF,82.24,-16.48,-151.75,light rain,
126,Biak,ID,84.06,-0.91,122.88,light rain,
130,Puerto Ayacucho,VE,76.03,5.66,-67.62,light rain,
134,Bouna,CI,76.89,9.27,-3.00,light rain,
137,Padang,ID,88.18,-0.95,100.35,light rain,


In [27]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [28]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
39,Kieta,PG,83.73,-6.22,155.63,light rain,
65,Butaritari,KI,83.77,3.07,172.79,light rain,Isles Sunset Lodge
72,Kavieng,PG,84.33,-2.57,150.80,light rain,Nusa Island Retreat
87,Sinnamary,GF,78.67,5.38,-52.95,moderate rain,ALETHEIA Appartement Sinnamary
102,Cayenne,GF,77.00,4.93,-52.33,moderate rain,Hôtel Le Dronmi
119,Faanui,PF,82.24,-16.48,-151.75,light rain,Bora-Bora Pearl Beach Resort & Spa
126,Biak,ID,84.06,-0.91,122.88,light rain,Kost Ariz
130,Puerto Ayacucho,VE,76.03,5.66,-67.62,light rain,Posada Turistica Manapiare
134,Bouna,CI,76.89,9.27,-3.00,light rain,Hotel Reserve
137,Padang,ID,88.18,-0.95,100.35,light rain,Mercure Padang


In [31]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

#Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [32]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")